# **Text Analysis: Using BERT to Predict a Company's Asset Intensity**
---
The goal of this notebook is to show the possibilities of predicting financial outcomes from textual data via DistilBERT transformer model.

This Colab notebook uses the following files:
*   `stock_des.csv` with Company descriptions of a large number of companues from Yahoo Finance.
*   `Assignment3 - Sneha Ekka.csv` with raw financial data downloaded from WRDS.

The steps this ipynb goes through are below:
1.   Upload `stock_des.csv`and `Assignment3 - Sneha Ekka.csv`
2.   Install and load the necessary libraries
3.   Load, merge, clean the data
4.   Create the label variable
5.   Prepare the predictor  
6.   Run the DistilBERT model
7.   Train logistic regression and evaluate its accuracy





## I. Install and Load the Libraries

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print('Device:', torch.device('cuda:0'))

Device: cuda:0


In [ ]:
# torch.cuda.get_device_name(0)

## II. Load Data Files

In [ ]:
# Load the csv files
stock_des = pd.read_csv('/content/stock_des.csv')
wrds_data = pd.read_csv('/content/Assignment3 - Sneha Ekka.csv')

In [ ]:
stock_des.head()

,ticker,description
0,AAPL,"Apple Inc. designs, manufactures, and markets ..."
1,MSFT,"Microsoft Corporation develops, licenses, and ..."
2,AMZN,"Amazon.com, Inc. engages in the retail sale of..."
3,FB,"Facebook, Inc. develops products that enable p..."
4,GOOGL,Alphabet Inc. provides online advertising serv...


In [ ]:
wrds_data.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,curcd,act,...,dt,ebitda,gp,icapt,invt,lct,sale,seq,wcap,costat
0,1004,2023-05-31,2022,INDL,C,D,STD,AIR,USD,1097.900,...,330.200,179.300,399.300,1417.000,624.700,351.500,1990.600,1099.100,746.40,A
1,1045,2023-12-31,2023,INDL,C,D,STD,AAL,USD,13572.000,...,40663.000,6267.000,14072.000,30520.000,2400.000,22062.000,52788.000,-5202.000,-8490.00,A
2,1075,2023-12-31,2023,INDL,C,D,STD,PNW,USD,1926.967,...,NaN,1618.683,1618.683,15035.673,493.547,2889.347,4695.991,6177.664,-962.38,A
3,1076,2023-12-31,2023,INDL,C,D,STD,PRG,USD,NaN,...,608.114,1849.328,2253.014,1199.445,0.000,NaN,2408.264,591.331,NaN,A
4,1078,2023-12-31,2023,INDL,C,D,STD,ABT,USD,22670.000,...,15873.000,10036.000,23507.000,53375.000,6570.000,13841.000,40109.000,38603.000,8829.00,A


In [ ]:
wrds_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2398 entries, 0 to 2397
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   gvkey     2398 non-null   int64  
 1   datadate  2398 non-null   object 
 2   fyear     2398 non-null   int64  
 3   indfmt    2398 non-null   object 
 4   consol    2398 non-null   object 
 5   popsrc    2398 non-null   object 
 6   datafmt   2398 non-null   object 
 7   tic       2398 non-null   object 
 8   curcd     2398 non-null   object 
 9   act       1824 non-null   float64
 10  at        2394 non-null   float64
 11  cogs      2394 non-null   float64
 12  dt        2074 non-null   float64
 13  ebitda    2271 non-null   float64
 14  gp        2394 non-null   float64
 15  icapt     2394 non-null   float64
 16  invt      2353 non-null   float64
 17  lct       1824 non-null   float64
 18  sale      2394 non-null   float64
 19  seq       2394 non-null   float64
 20  wcap      1824 non-null   floa

## III. Clean, Merge, Prepare the Data

In [ ]:
# Clean up the column names
wrds_data.rename(columns={'tic':'ticker'}, inplace=True)

# Drop columns that we don't need
wrds_data.drop(columns=['gvkey','indfmt','consol','popsrc','datafmt','curcd','costat'], inplace=True)

In [ ]:
# Merge the dataframes into one

df = pd.merge(wrds_data, stock_des, on='ticker')
df.head()

,datadate,fyear,ticker,act,at,cogs,dt,ebitda,gp,icapt,invt,lct,sale,seq,wcap,description
0,2023-05-31,2022,AIR,1097.900,1833.100,1591.300,330.200,179.300,399.300,1417.000,624.700,351.500,1990.600,1099.100,746.40,AAR Corp. provides products and services to co...
1,2023-12-31,2023,AAL,13572.000,63058.000,38716.000,40663.000,6267.000,14072.000,30520.000,2400.000,22062.000,52788.000,-5202.000,-8490.00,"American Airlines Group Inc., through its subs..."
2,2023-12-31,2023,PNW,1926.967,24661.153,3077.308,NaN,1618.683,1618.683,15035.673,493.547,2889.347,4695.991,6177.664,-962.38,"Pinnacle West Capital Corporation, through its..."
3,2023-12-31,2023,PRG,NaN,1491.255,155.250,608.114,1849.328,2253.014,1199.445,0.000,NaN,2408.264,591.331,NaN,"PROG Holdings, Inc. operates as an omnichannel..."
4,2023-12-31,2023,ABT,22670.000,73214.000,16602.000,15873.000,10036.000,23507.000,53375.000,6570.000,13841.000,40109.000,38603.000,8829.00,"Abbott Laboratories discovers, develops, manuf..."


In [ ]:
# Checking the shape of our dataframes
print("Stock Description:", stock_des.shape)
print("WRDS Data:", wrds_data.shape)
print("Final merged data:", df.shape)

Stock Description: (2852, 2)
WRDS Data: (2398, 15)
Final merged data: (2390, 16)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390 entries, 0 to 2389
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   datadate     2390 non-null   object 
 1   fyear        2390 non-null   int64  
 2   ticker       2390 non-null   object 
 3   act          1816 non-null   float64
 4   at           2386 non-null   float64
 5   cogs         2386 non-null   float64
 6   dt           2066 non-null   float64
 7   ebitda       2263 non-null   float64
 8   gp           2386 non-null   float64
 9   icapt        2386 non-null   float64
 10  invt         2345 non-null   float64
 11  lct          1816 non-null   float64
 12  sale         2386 non-null   float64
 13  seq          2386 non-null   float64
 14  wcap         1816 non-null   float64
 15  description  2390 non-null   object 
dtypes: float64(12), int64(1), object(3)
memory usage: 298.9+ KB


In [ ]:
# Checking for NULLs in the data
nan_counts = df.isnull().sum()
nan_percent = df.isnull().sum()/df.shape[0]
nans_dict = {'count_of_nans':nan_counts, 'percent_of_nans':nan_percent}

pd.DataFrame(nans_dict).sort_values('percent_of_nans')

,count_of_nans,percent_of_nans
datadate,0,0.000000
fyear,0,0.000000
ticker,0,0.000000
description,0,0.000000
at,4,0.001674
cogs,4,0.001674
gp,4,0.001674
icapt,4,0.001674
sale,4,0.001674
seq,4,0.001674


In [ ]:
# Creating the financial ratios

# Ratio 1
df['gp_ass'] = df['gp'] / df['at']

# Ratio 2
df['gp_margin'] = df['gp'] / df['sale']

# Ratio 3
df['dt_at'] = df['dt'] / df['at']

In [ ]:
# Checking for NULLs in the data
nan_counts = df.isnull().sum()
nan_percent = df.isnull().sum()/df.shape[0]
nans_dict = {'count_of_nans':nan_counts, 'percent_of_nans':nan_percent}

pd.DataFrame(nans_dict).sort_values('percent_of_nans')

,count_of_nans,percent_of_nans
datadate,0,0.000000
fyear,0,0.000000
ticker,0,0.000000
description,0,0.000000
gp_ass,4,0.001674
seq,4,0.001674
sale,4,0.001674
gp,4,0.001674
icapt,4,0.001674
cogs,4,0.001674


**Note:** Will deal with NULL values in the following cells when we work with the individual ratios.

In [ ]:
# Define the list of financial ratios
ratios = ['gp_ass', 'gp_margin', 'dt_at']

# Create a separate plot for each ratio
for ratio in ratios:
    # Create a new figure for each ratio
    fig = go.Figure()

    # Add box plot for the current ratio
    fig.add_trace(go.Box(y=df[ratio], name=ratio))

    # Update layout for the current plot
    fig.update_layout(title=f'Boxplot of {ratio}',
                      xaxis=dict(title='Ratio'),
                      yaxis=dict(title=ratio))

    # Show the current plot
    fig.show()

In [ ]:
# Filtering out extreme outliers using the Boxplots plotted above

df_1 = df[(df['gp_ass'] >= -1) & (df['gp_ass'] <= 1.5)][['ticker','description','gp_ass']]
print("Gross Profit to Asset dataframe:", df_1.shape)

df_2 = df[df['gp_margin'] >= -500][['ticker','description','gp_margin']]
print("Gross Profit Margin dataframe:", df_2.shape)

df_3 = df[df['dt_at'] <= 1.5][['ticker','description','dt_at']]
print("Debt to Asset dataframe:", df_3.shape)

Gross Profit to Asset dataframe: (2377, 3)
Gross Profit Margin dataframe: (2322, 3)
Debt to Asset dataframe: (2053, 3)


Create a binary variable that is 1 if the assets to revenue ratio is above its median and 0 otherwise. This is the **dependent variable** (label) that we'll try to predict.

In [ ]:
df_1['HIGH_GPAS'] = (df_1['gp_ass'].gt(df_1['gp_ass'].median())).astype(int)
df_2['HIGH_GPM'] = (df_2['gp_margin'].gt(df_2['gp_margin'].median())).astype(int)
df_3['HIGH_DTAT'] = (df_3['dt_at'].gt(df_3['dt_at'].median())).astype(int)

## IV. Preparing the Text Data

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Count the number of tokens in the 'text' column
def get_tokens(df):
    df['token_count'] = df['description'].apply(lambda x: len(word_tokenize(x)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Getting tokens for each of the 3 dataframes with 3 ratios
dataframes = [df_1, df_2, df_3]

for i, df in enumerate(dataframes, start=1):
    get_tokens(df)
    print(f"Ratio {i}: {df.shape}")

Ratio 1: (2377, 5)
Ratio 2: (2322, 5)
Ratio 3: (2053, 5)


In [ ]:
import re

# Function to remove unusual characters
def remove_unusual_characters(text):
    pattern = r'[^a-zA-Z0-9.,?! ]'  # Pattern to match common English characters, numbers, and punctuation
    text = re.sub(pattern, '', text)
    return text

# Loop through each DataFrame
for i, df in enumerate(dataframes, start=1):
    # Apply the function to the 'description' column
    df['text'] = df['description'].apply(remove_unusual_characters)
    print(f"DataFrame {i} processed.")

DataFrame 1 processed.
DataFrame 2 processed.
DataFrame 3 processed.


In [ ]:
from nltk.tokenize import sent_tokenize

# Remove very long descriptions
def filter_token_count(df):

    lower_threshold = df['token_count'].quantile(0.00)
    upper_threshold = df['token_count'].quantile(0.99)

    # Trim the DataFrame based on the 1% thresholds
    trimmed_df = df[(df['token_count'] >= lower_threshold) & (df['token_count'] <= upper_threshold)]

    trimmed_df=trimmed_df[trimmed_df["ticker"]!="LLY"] # these 2 have special characters in description, hence removed
    trimmed_df=trimmed_df[trimmed_df["ticker"]!="BIIB"]
    trimmed_df=trimmed_df.reset_index(drop=True)

    return trimmed_df


# Function to keep first sentence
def keep_n_sentences(text):
    sentences = sent_tokenize(text)
    first_three_sentences = sentences[:1]
    return ' '.join(first_three_sentences)

In [ ]:
# Applying the above functions to the 3 dataframes
trimmed_df1 = filter_token_count(df_1)
trimmed_df1['text'] = trimmed_df1['text'].apply(keep_n_sentences)

trimmed_df2 = filter_token_count(df_2)
trimmed_df2['text'] = trimmed_df2['text'].apply(keep_n_sentences)

trimmed_df3 = filter_token_count(df_3)
trimmed_df3['text'] = trimmed_df3['text'].apply(keep_n_sentences)

## V. Preparing the predictor and DistilBERT model

**Note**. Please enable GPU in Edit > Notebook settings > Hardware accelerator.

Load a pre-trained BERT model.

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, max_length=2)
model = model_class.from_pretrained(pretrained_weights)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Tokenize the textual data for DistilBERT.

In [ ]:
tokenized_1 = trimmed_df1['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_2 = trimmed_df2['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_3 = trimmed_df3['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Pad all lists of tokenized values to the same size.

In [ ]:
max_len = 0
for i in tokenized_1.values:
    if len(i) > max_len:
        max_len = len(i)

padded_1 = np.array([i + [0]*(max_len-len(i)) for i in tokenized_1.values])

ax_len = 0
for i in tokenized_2.values:
    if len(i) > max_len:
        max_len = len(i)

padded_2 = np.array([i + [0]*(max_len-len(i)) for i in tokenized_2.values])

max_len = 0
for i in tokenized_3.values:
    if len(i) > max_len:
        max_len = len(i)

padded_3 = np.array([i + [0]*(max_len-len(i)) for i in tokenized_3.values])

In [ ]:
# Create attention mask variable for BERT to ignore (mask) the padding when it's processing its input
attention_mask_1 = np.where(padded_1 != 0, 1, 0)
print("Ratio 1:", attention_mask_1.shape)

attention_mask_2 = np.where(padded_2 != 0, 1, 0)
print("Ratio 2:", attention_mask_2.shape)

attention_mask_3 = np.where(padded_3 != 0, 1, 0)
print("Ratio 3:", attention_mask_3.shape)

Ratio 1: (2351, 135)
Ratio 2: (2296, 135)
Ratio 3: (2030, 135)


## VI. DistilBERT Model

We run the pretrained DistilBERT model on the prepared predictor and keep the result in `last_hidden_states` variable.

In [ ]:
input_ids_1 = torch.tensor(padded_1)
attention_mask_1 = torch.tensor(attention_mask_1)

with torch.no_grad():
    last_hidden_states_1 = model(input_ids_1,
                                 attention_mask=attention_mask_1)

In [ ]:
input_ids_2 = torch.tensor(padded_2)
attention_mask_2 = torch.tensor(attention_mask_2)

with torch.no_grad():
    last_hidden_states_2 = model(input_ids_2,
                                 attention_mask=attention_mask_2)

In [ ]:
input_ids_3 = torch.tensor(padded_3)
attention_mask_3 = torch.tensor(attention_mask_3)

with torch.no_grad():
    last_hidden_states_3 = model(input_ids_3,
                                 attention_mask=attention_mask_3)

## VII. Logistic Regression Model



Keep the first layer of the hidden states and assign the outcome variable to `labels`.

### i. Ratio 1 - Gross Profit to Assets Ratio

In [ ]:
features_1 = last_hidden_states_1[0][:,0,:].numpy()
labels_1 = trimmed_df1['HIGH_GPAS']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features_1, labels_1, random_state=42)
lr_clf_1 = LogisticRegression(max_iter=5000)
lr_clf_1.fit(train_features, train_labels)
print(lr_clf_1.score(test_features, test_labels))

0.7772108843537415


In [ ]:
clf_1 = DummyClassifier()

scores = cross_val_score(clf_1, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.518 (+/- 0.00)


**Gross Profit to Assets Ratio:**

- The gross profit to assets ratio measures a company's ability to generate gross profit from its total assets. It is calculated by dividing the company's gross profit (revenue minus cost of goods sold) by its total assets. A higher ratio indicates that a company is efficiently converting its assets into profitable sales, while a lower ratio may suggest underutilization of assets or operation in a low-margin industry.
- The acceptable level of the gross profit to assets ratio can vary significantly across different industries. Companies with capital-intensive operations tend to have lower ratios compared to more asset-light businesses. Tracking changes in a company's ratio over time provides insights into its operational efficiency and profitability, with an improving ratio signaling better asset utilization and a declining ratio potentially indicating issues with cost control or pricing power.

**Perfomance Evaluation:**
- The logistic regression model achieves an accuracy of approximately 77.72% on the test set. A high accuracy indicates that the model is effective at distinguishing between high and low Gross Profit to Assets ratio based on textual descriptions. Additionally, the dummy classifier serves as a baseline model that predicts classes randomly or using a simple rule. If the model score is greater than dummy classifier score (0.518), then the model makes correct predictions.

### ii. Ratio 2 - Gross Profit Margin

In [ ]:
features_2 = last_hidden_states_2[0][:,0,:].numpy()
labels_2 = trimmed_df2['HIGH_GPM']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features_2, labels_2, random_state=42)
lr_clf_2 = LogisticRegression(max_iter=5000)
lr_clf_2.fit(train_features, train_labels)
print(lr_clf_2.score(test_features, test_labels))

0.7613240418118467


In [ ]:
clf_2 = DummyClassifier()

scores = cross_val_score(clf_2, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.502 (+/- 0.00)


**Gross Profit Margin:**
- The gross profit margin ratio measures a company's efficiency by dividing its gross profit (revenue minus cost of goods sold) by total revenue. This ratio represents the percentage of each sales dollar retained as gross profit after accounting for direct production costs. A higher gross margin suggests effective pricing, cost control, or operation in a high-margin industry, while a lower ratio may indicate pricing pressure, high input costs, or operational inefficiencies.
- The acceptable gross profit margin can vary significantly across industries. Companies in sectors with high barriers to entry, such as technology or pharmaceuticals, tend to have higher margins compared to low-margin, high-volume businesses like retail or commodities. Tracking changes in a company's gross profit margin over time provides insights into its pricing power, cost management, and overall profitability.

**Perfomance Evaluation:**
- The logistic regression model achieves an accuracy of approximately 76.13% on the test set. This accuracy is relatively lesser than the previous ratio but indicates that the model is still effective at distinguishing between high and low Gross Profit Margin ratio based on textual descriptions. Additionally, the dummy classifier serves as a baseline model that predicts classes randomly or using a simple rule. If the model score is greater than dummy classifier score (0.502), then the model makes correct predictions.

### iii. Ratio 3 - Debt to Asset Ratio

In [ ]:
features_3 = last_hidden_states_3[0][:,0,:].numpy()
labels_3 = trimmed_df3['HIGH_DTAT']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features_3, labels_3, random_state=42)
lr_clf_3 = LogisticRegression(max_iter=5000)
lr_clf_3.fit(train_features, train_labels)
print(lr_clf_3.score(test_features, test_labels))

0.6003937007874016


In [ ]:
clf_3 = DummyClassifier()

scores = cross_val_score(clf_3, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.509 (+/- 0.00)


**Debt to Asset Ratio**

- The debt to asset ratio measures the proportion of a company's total assets that are financed by debt. It is calculated by dividing a company's total liabilities by its total assets. This ratio provides insight into a company's capital structure and financial leverage.
- A higher debt to asset ratio indicates that a company is more heavily financed by debt, which can increase financial risk but may also provide tax benefits and financial flexibility. Conversely, a lower ratio suggests the company relies more on equity financing and is less leveraged. The acceptable level of debt to asset ratio can vary significantly by industry, as capital-intensive businesses often have higher ratios than asset-light industries. Tracking changes in a company's debt to asset ratio over time can reveal shifts in its financing strategy and risk profile, which is important for investors and creditors to assess the company's financial stability and long-term sustainability.

**Perfomance Evaluation:**
- The logistic regression model achieves an accuracy of approximately 60% on the test set. This accuracy is the least as compared to the previous two ratios. However, the model score is greater than dummy classifier score (0.509), and indicates that the model makes correct predictions.